In [1]:
# Imports
import sympy as sp

p_s, v_s, a_s = sp.symbols(r'\vec{p}_s \vec{v}_s \vec{a}_s')
k, n = sp.symbols(r'k n')
del_t = sp.Symbol(r'\Delta t')
xi, eta = sp.symbols(r'\xi \eta')
p_k, v_k, a_k = sp.symbols(r'\vec{p}_k \vec{v}_k \vec{a}_k')
phi_k = sp.Function(r'\phi_k')(xi, eta)

\pagebreak

# Introduction of the Problem

Consider a patch and node that moves with time. Our goal is to determine the contact point in the reference space of the patch/element. The mapping of a reference point $(\xi, \eta)$ to the global space is given by

$$\vec{s}=\sum_{p=0}^{n-1}\phi_p(\xi, \eta)\vec{s}_p$$

where $\vec{s}$ is the position in the global space and $\vec{s}_p$ is a basis vector of the patch in the global space. If we consider the contact point to be of interest ($\vec{s}_c$) and the fact that its position moves with time as well as the basis vectors, we can write

$$\vec{p}_s + \vec{v}_s\Delta t + \frac{1}{2}\vec{a}_s\Delta t^2=\sum_{k=0}^{n-1}\phi_k(\xi,\eta)(\vec{p}_k + \vec{v}_k\Delta t + \frac{1}{2}\vec{a}_k\Delta t^2)$$

where $p$, $v$, and $a$ are the position, velocity, and acceleration vectors, respectively. The subscript $s$ denotes the slave node, and the subscript $k$ denotes the patch nodes. This relationship can be re-written as a matrix multiplication like so

$$
\vec{p}_s + \vec{v}_s\Delta t + \frac{1}{2}\vec{a}_s\Delta t^2 = 
\underbrace{
\begin{bmatrix}
p_{x0}+v_{x0}\Delta t + \frac{1}{2}a_{x0}\Delta t^2 & p_{x1}+v_{x1}\Delta t + \frac{1}{2}a_{x1}\Delta t^2 & \cdots \\
p_{y0}+v_{y0}\Delta t + \frac{1}{2}a_{y0}\Delta t^2 & p_{y1}+v_{y1}\Delta t + \frac{1}{2}a_{y1}\Delta t^2 & \cdots \\
p_{z0}+v_{z0}\Delta t + \frac{1}{2}a_{z0}\Delta t^2 & p_{z1}+v_{z1}\Delta t + \frac{1}{2}a_{z1}\Delta t^2 & \cdots
\end{bmatrix}
}_{A}
\begin{bmatrix}
\phi_0(\xi,\eta) \\
\phi_1(\xi,\eta) \\
\vdots \\
\phi_{n - 1}(\xi,\eta)
\end{bmatrix}
$$

The above results in a system of three equations and three unknowns - $\xi$, $\eta$, and $\Delta t$.

After the contact point is found as well as the $\Delta t$, the node is considered to be in contact with the patch if the following conditions are met:

1. All reference coordinates ($\xi$, $\eta$) are between -1 and 1.
2. The solution for $\Delta t$ is between 0 and $dt$ (the time step of the explicit analysis).

# Sympy Solution

In [2]:
eq1 = sp.Eq(p_s + v_s*del_t + sp.Rational(1, 2)*a_s*del_t**2, sp.Sum(phi_k*(p_k + v_k*del_t + sp.Rational(1, 2)*a_k*del_t**2), (k, 0, n-1)))
eq1

Eq(\Delta t**2*\vec{a}_s/2 + \Delta t*\vec{v}_s + \vec{p}_s, Sum((\Delta t**2*\vec{a}_k/2 + \Delta t*\vec{v}_k + \vec{p}_k)*\phi_k(\xi, \eta), (k, 0, n - 1)))

In [3]:
# Working off the matrix form
A = sp.Matrix([sp.Function('A')(del_t)])
eq2 = sp.Eq(eq1.lhs, sp.MatMul(A, sp.Matrix([phi_k])), evaluate=False)
eq2

Eq(\Delta t**2*\vec{a}_s/2 + \Delta t*\vec{v}_s + \vec{p}_s, Matrix([[A(\Delta t)]])*Matrix([[\phi_k(\xi, \eta)]]))

In the vector form, the Newton-Raphson scheme is

$$
\begin{bmatrix}
\xi_{i+1} \\
\eta_{i+1} \\
\Delta t_{i+1} 
\end{bmatrix} = \begin{bmatrix}
\xi_{i} \\
\eta_{i} \\
\Delta t_{i}
\end{bmatrix} - \bf{J^{-1}F}
$$

In [4]:
# Constructing the vector function F
F = eq2.rhs - sp.Matrix([eq2.lhs])
F

Matrix([[-\Delta t**2*\vec{a}_s/2 - \Delta t*\vec{v}_s - \vec{p}_s + A(\Delta t)*\phi_k(\xi, \eta)]])

In [5]:
# Constructing the Jacobian matrix J
J = F.jacobian([xi, eta, del_t])
J

Matrix([[A(\Delta t)*Derivative(\phi_k(\xi, \eta), \xi), A(\Delta t)*Derivative(\phi_k(\xi, \eta), \eta), -\Delta t*\vec{a}_s - \vec{v}_s + \phi_k(\xi, \eta)*Derivative(A(\Delta t), \Delta t)]])

The above is correct, but the $\frac{\partial}{\partial\Delta t}A(\Delta_t)$ must be to the left of the $\phi_k$ vector. The $\frac{\partial}{\partial \Delta t}A$ matrix is constructed as 

$$
\frac{\partial}{\partial \Delta t}A = \begin{bmatrix}
v_{x0} + a_{x0}\Delta t & v_{x1} + a_{x1}\Delta t & \cdots \\
v_{y0} + a_{y0}\Delta t & v_{y1} + a_{y1}\Delta t & \cdots \\
v_{z0} + a_{z0}\Delta t & v_{z1} + a_{z1}\Delta t & \cdots
\end{bmatrix}
$$